# Final Project
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__  
Throughout this course you’ve engaged with key principles required to develop scalable machine learning analyses for structured and unstructured data. Working in Hadoop Streaming and Spark you’ve learned to translate common machine learning algorithms into Map-Reduce style implementations. You’ve developed the ability to evaluate Machine Learning approaches both in terms of their predictive performance as well as their scalability. For the final project you will demonstrate these skills by solving a machine learning challenge on a new dataset. Your job is to perform Click Through Rate prediction on a large dataset of Criteo advertising data made public as part of a Kaggle competition a few years back. As you perform your analysis, keep in mind that we are not grading you on the final performance of your model or how ‘advanced’ the techniques you use but rather on your ability to explain and develop a scalable machine learning approach to answering a real question.

More about the dataset:
https://www.kaggle.com/c/criteo-display-ad-challenge

# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [26]:
import re
import ast
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import col,sum,when,isnan,count
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import time


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
# start Spark Session
app_name = "fproj_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [5]:
# create a data directory (RUN THIS CELL AS IS)
# !mkdir data

In [115]:
# grab the tar.gz file from kaggle

# !curl https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz -o data/dac.tar.gz

In [116]:
# I couldn't get this to unpack the tarball, so I just did it in Windows
# !tar -xvz data/dac.tar.gz

In [117]:
# take a look at the to the training data set
# !head data/train.txt

In [118]:
# ! ls ../Assignments/FinalProject/

In [33]:
# load the data into Spark RDD for convenience of use later (RUN THIS CELL AS IS)
projectRDD = sc.textFile('../Assignments/FinalProject/data/train.txt')
# projectRDD = projectRDD.sample(False,.001)
# sampleRDD = sc.textFile('../Assignments/FinalProject/data/train1-500.txt')
projectRDD.count()

45840617

__`REMINDER:`__ If you are running this notebook on the course docker container, you can monitor the progress of your jobs using the Spark UI at: http://localhost:4040/jobs/

# Question 1: Question Formulation 
Introduce the goal of your analysis. What questions will you seek to answer, why do people perform this kind of analysis on this kind of data? Preview what level of performance your model would need to achieve to be practically useful.

# Question 2: Algorithm Explanation
Create your own toy example that matches the dataset provided and use this toy example to explain the math behind the algorithym that you will perform.

# Question 3: EDA & Discussion of Challenges
Determine 2-3 relevant EDA tasks that will help you make decisions about how you implement the algorithm to be scalable. Discuss any challenges that you anticipate based on the EDA you perform

In [34]:
sqlContext = SQLContext(sc)

# Load a text file and convert each line to a Row.
# lines = sc.textFile("../Assignments/FinalProject/data/train.txt")

projectRDD = projectRDD.map(lambda l: l.split("\t"))\
                        .map(lambda p: Row(label=int(p[0]), I1=p[1], I2=p[2],\
                        I3=p[3], I4=p[4], I5=p[5], I6=p[6],\
                        I7=p[7], I8=p[8], I9=p[9], I10=p[10],\
                        I11=p[11], I12=p[12], I13=p[13], C1=p[14], C2=p[15], C3=p[16],\
                        C4=p[17], C5=p[18], C6=p[19], C7=p[20], C8=p[21], C9=p[22],\
                        C10=p[23], C11=p[24], C12=p[25], C13=p[26], C14=p[27], C15=p[28],\
                        C16=p[29], C17=p[30], C18=p[31], C19=p[32], C20=p[33], C21=p[34],\
                        C22=p[35], C23=p[36], C24=p[37], C25=p[38], C26=p[39]))

# Infer the schema, and register the DataFrame as a table.
projectDF = sqlContext.createDataFrame(projectRDD)
projectDF.registerTempTable("projectTable")


In [35]:
projectDF.printSchema()
projectDF.head()

root
 |-- C1: string (nullable = true)
 |-- C10: string (nullable = true)
 |-- C11: string (nullable = true)
 |-- C12: string (nullable = true)
 |-- C13: string (nullable = true)
 |-- C14: string (nullable = true)
 |-- C15: string (nullable = true)
 |-- C16: string (nullable = true)
 |-- C17: string (nullable = true)
 |-- C18: string (nullable = true)
 |-- C19: string (nullable = true)
 |-- C2: string (nullable = true)
 |-- C20: string (nullable = true)
 |-- C21: string (nullable = true)
 |-- C22: string (nullable = true)
 |-- C23: string (nullable = true)
 |-- C24: string (nullable = true)
 |-- C25: string (nullable = true)
 |-- C26: string (nullable = true)
 |-- C3: string (nullable = true)
 |-- C4: string (nullable = true)
 |-- C5: string (nullable = true)
 |-- C6: string (nullable = true)
 |-- C7: string (nullable = true)
 |-- C8: string (nullable = true)
 |-- C9: string (nullable = true)
 |-- I1: string (nullable = true)
 |-- I10: string (nullable = true)
 |-- I11: string (nullabl

Row(C1='68fd1e64', C10='a8cd5504', C11='b2cb9c98', C12='37c9c164', C13='2824a5f6', C14='1adce6ef', C15='8ba8b39a', C16='891b62e7', C17='e5ba7672', C18='f54016b9', C19='21ddcdc9', C2='80e26c9b', C20='b1252a9d', C21='07b5194c', C22='', C23='3a171ecb', C24='c5c50484', C25='e8b83407', C26='9727dd16', C3='fb936136', C4='7b4723c4', C5='25c83c98', C6='7e0ccccf', C7='de7995b8', C8='1f89b562', C9='a73ee510', I1='1', I10='1', I11='2', I12='', I13='2', I2='1', I3='5', I4='0', I5='1382', I6='4', I7='15', I8='2', I9='181', label=0)

In [36]:
projectDF = projectDF.withColumn("I1", projectDF["I1"].cast("int"))
projectDF = projectDF.withColumn("I2", projectDF["I2"].cast("int"))
projectDF = projectDF.withColumn("I3", projectDF["I3"].cast("int"))
projectDF = projectDF.withColumn("I4", projectDF["I4"].cast("int"))
projectDF = projectDF.withColumn("I5", projectDF["I5"].cast("int"))
projectDF = projectDF.withColumn("I6", projectDF["I6"].cast("int"))
projectDF = projectDF.withColumn("I7", projectDF["I7"].cast("int"))
projectDF = projectDF.withColumn("I8", projectDF["I8"].cast("int"))
projectDF = projectDF.withColumn("I9", projectDF["I9"].cast("int"))
projectDF = projectDF.withColumn("I10", projectDF["I10"].cast("int"))
projectDF = projectDF.withColumn("I11", projectDF["I11"].cast("int"))
projectDF = projectDF.withColumn("I12", projectDF["I12"].cast("int"))
projectDF = projectDF.withColumn("I13", projectDF["I13"].cast("int"))

In [37]:
projectDF.count()

45840617

In [38]:
# numRows = projectRDD.count()
# numRows

In [39]:
# # get a sample to do things at a small scale with
# sampleSize = 10000 # change this as we get sense of compute times
# sampleDF = projectDF.sample(withReplacement = False, fraction = sampleSize/numRows, seed = 1000) 
# sampleDF.createOrReplaceTempView("sampleDFTable") # I don't really know what this does

In [40]:
def handle_missing(data):
    """
    Replaces missiong values ('',' ',NA, NaN) with specified value
    Args:
        data - Spark DF with missing values
        filler - Value to replace missing with defaults to 0
    Returns:
        data - Spark DF with missing vales filled in
    """
    
    for c in data.columns:
        data = data.replace(' ',str('EMPTY'), c)
        data = data.replace('',str('EMPTY'), c)
    data = data.na.fill(0)
    return data

def pipeline_stages(categorical_features,numeric_features):
    """Constructs a list of pipeline stages which will transform
       All categorical features into one-hot vectors
       
       Args:
           categorical_features - list of categorical feature column names
           numeric_features - list of numeric feature column names
       Returns: 
           stages - list of pipeline stages which will convert categorical features to
               one-hot vectors
        """
    stages = []
    for cat_col in categorical_features:
        stringIndexer = StringIndexer(inputCol = cat_col, outputCol = cat_col + 'Index')
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[cat_col + "classVec"],handleInvalid='keep')
        stages += [stringIndexer, encoder]

    # label_stringIdx = StringIndexer(inputCol = 'label', outputCol = 'label')
    # stages += [label_stringIdx]

    assemblerInputs = [c + "classVec" for c in categorical_features] + numeric_features
    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    stages += [assembler]
    
    return stages
def apply_pipeline(data,stages):
    """
    Applies pipeline stages to data
    Args: 
        data - Spark DF of data to be modified
        stages - list of pipeline stages to be applied to data
    returns:
        preppedDataDF - Spark DF with variable transformations applied

    """
    partialPipeline = Pipeline().setStages(stages)
    pipelineModel = partialPipeline.fit(data)
    preppedDataDF = pipelineModel.transform(data)

    
    return preppedDataDF

In [41]:
numeric_features = ["I1", "I2", "I3", "I4", "I5", "I6", "I7", "I8", "I9", "I10", "I11", "I12", "I13"]

In [42]:
categorical_features =  ['C6','C9','C14','C17','C20','C22','C23']
# ["C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10", "C11", "C12",\
#                         "C13", "C14", "C15", "C16", "C17", "C18", "C19", "C20", "C21", "C22", "C23",\
#                        "C24", "C25", "C26"]

In [43]:
label_and_numeric = ["label", "I1", "I2", "I3", "I4", "I5", "I6", "I7", "I8", "I9", "I10", "I11", "I12", "I13"]

## Prep the Data

In [44]:
weights = [.8, .2]
seed = 4

stages = []
stages = pipeline_stages(categorical_features,numeric_features)

keep_cols = categorical_features + label_and_numeric
projectDF = projectDF.select([c for c in projectDF.columns if c in keep_cols])
#Handle empty strings by imputing 'EMPTY'
for c in projectDF.columns:
    projectDF = projectDF.replace(' ',str('EMPTY'), c)
    projectDF = projectDF.replace('',str('EMPTY'), c)
#Set NA's to 0
projectDF = projectDF.na.fill(0)

#OHE -> VectorAssembler
preppedDF = apply_pipeline(projectDF,stages)
train,test = preppedDF.randomSplit(weights,seed)

In [45]:
print('Train Size - ',train.count())
print('Test Size - ',test.count())

print('Train Label Distribution -',train.groupBy('label').count().show())
print('Test Label Distribution -',test.groupBy('label').count().show())

Train Size -  36677888
Test Size -  9162729
+-----+--------+
|label|   count|
+-----+--------+
|    0|27279208|
|    1| 9398680|
+-----+--------+

Train Label Distribution - None
+-----+-------+
|label|  count|
+-----+-------+
|    0|6815971|
|    1|2346758|
+-----+-------+

Test Label Distribution - None


## Logistic Regression

In [46]:
#Logistic Regression
t_lr = time.time()
lrModel = LogisticRegression().fit(train)
predictions_lr = lrModel.transform(test)
# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction",labelCol="label")
print('AUC ROC', evaluator.evaluate(predictions_lr))
print('Total Time -',time.time()-t_lr)

Py4JJavaError: An error occurred while calling o3130.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 33 in stage 958.0 failed 1 times, most recent failure: Lost task 33.0 in stage 958.0 (TID 160413, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:52)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:156)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.getSortedIterator(UnsafeExternalSorter.java:477)
	at org.apache.spark.sql.execution.UnsafeExternalRowSorter.sort(UnsafeExternalRowSorter.java:168)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 47796)
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/opt/anaconda/lib/python3.6/site-packages/pyspark-2.3.1-py3.6.egg/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/opt/anaconda/lib/python3.6/site-packages/pyspark-2.3.1-py3.6.egg/pyspark/serializers.py", line 685, in read_int
    raise EOFError
EOFError
---------------------------------

In [ ]:
#LR Predictions
predictions_lr.groupBy('prediction').count().show()

## Decision Tree

In [ ]:
# Create initial Decision Tree Model
t_dt = time.time()
dtModel = DecisionTreeClassifier(
    labelCol="label", featuresCol="features", maxDepth=3).fit(train)

#make predictions
predictions_dt = dtModel.transform(test)
evaluator = BinaryClassificationEvaluator(
    rawPredictionCol="prediction",labelCol="label")
print('AUC ROC -',evaluator.evaluate(predictions_dt))
print(time.time()-t_dt)

In [ ]:
# DT Predictions
predictions_dt.groupBy('prediction').count().show()

## Random Forest

In [ ]:
# Create an initial RandomForest model.
t_rf = time.time()
rfModel = RandomForestClassifier(
    labelCol="label", featuresCol="features").fit(train)
#Make Predictions
predictions_rf = rfModel.transform(test)
evaluator = BinaryClassificationEvaluator(
    rawPredictionCol="prediction",labelCol="label")
print('AUC ROC -',evaluator.evaluate(predictions_rf))
print(time.time()-t_rf)

In [ ]:
#RF Predictions
predictions_rf.groupBy('prediction').count().show()

In [25]:
# #Convert to Numpy Objects NOT FEASIBLE AT SCALE
# pred_lr = np.array(predictions_lr.select('prediction').collect())
# pred_dt = np.array(predictions_dt.select('prediction').collect())
# pred_rf = np.array(predictions_rf.select('prediction').collect())
# y = np.array(test.select('label').collect())
# print('Accuracy LR -',np.sum(y == pred_lr) / len(y))
# print('Accuracy dt -',np.sum(y == pred_dt) / len(y))
# print('Accuracy rf -',np.sum(y == pred_rf) / len(y))

Accuracy LR - 0.7567185289957568
Accuracy dt - 0.7476879556087477
Accuracy rf - 0.748993580676749


# Question 4: Algorithm Implementation 
Develop a 'homegrown' implementation of the algorithn, apply it to the training dataset and evaluate your results on the test set. 

# Question 5: Application of Course Concepts
Pick 3-5 key course concepts and discuss how your work on this assignment illustrates an understanding of these concepts. 